In [24]:
import nltk
from nltk.tokenize import word_tokenize #tokenize word
from nltk.stem import WordNetLemmatizer #run ran running => make same actual word
import numpy as np
import random
import pickle #save file
from collections import Counter #count word

In [22]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shaha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shaha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [12]:
lemmatizer = WordNetLemmatizer()
hm_lines = 100000

In [13]:
def create_lexicon(pos,neg):

    lexicon = []
    with open(pos,'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            all_words = word_tokenize(l)
            lexicon += list(all_words)

    with open(neg,'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            all_words = word_tokenize(l)
            lexicon += list(all_words)

In [44]:
def sample_handling(sample,lexicon,classification):

    featureset = []

    with open(sample,'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            current_words = word_tokenize(l.lower())
            current_words = [lemmatizer.lemmatize(i) for i in current_words]
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    features[index_value] += 1

            features = list(features)
            featureset.append([features,classification])

    return featureset


In [45]:
def create_feature_sets_and_labels(pos,neg,test_size = 0.1):
    lexicon = create_lexicon(pos,neg)
    features = []
    features += sample_handling('pos.txt',lexicon,[1,0])
    random.shuffle(features)
    features += sample_handling('neg.txt',lexicon,[0,1])
    random.shuffle(features)
    features = np.array(features)

    testing_size = int(test_size*len(features))

    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])

    return train_x,train_y,test_x,test_y

In [46]:
if __name__ == '__main__':
    train_x,train_y,test_x,test_y = create_feature_sets_and_labels('pos.txt','neg.txt')

    with open('sentiment_set.pickle','wb') as f:
        pickle.dump([train_x,train_y,test_x,test_y],f)

423


# ANN

In [48]:
import tensorflow as tf
import pickle
import numpy as np

In [49]:
n_nodes_hl1 = 1500
n_nodes_hl2 = 1500
n_nodes_hl3 = 1500

In [50]:
n_classes = 2
batch_size = 100
hm_epochs = 10

In [58]:
x = tf.placeholder('float')
Y = tf.placeholder('float')

In [52]:
hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

hidden_3_layer = {'f_fum':n_nodes_hl3,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}

output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}

In [53]:
def neural_network_model(data):

    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weight']), hidden_3_layer['bias'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weight']) + output_layer['bias']

    return output

In [81]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=Y) )
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    n_epochs = 30
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer() )
        
        for epoch in range(n_epochs):
            epoch_loss = 0
            
            i=0
            while i < len(train_x):
                start = i
                end = i+batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                              Y: batch_y})
                epoch_loss += c
                i+=batch_size
            
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss,
                  'With Accuracy:',accuracy.eval({x:test_x,Y:test_y}))

        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('\n\n','#'*50,'\n Total Accuracy:',accuracy.eval({x:test_x,Y:test_y}),'\n','#'*50)

In [82]:
train_neural_network(x)

Epoch 1 completed out of 10 loss: 1253146.99121 With Accuracy: 0.589118
Epoch 2 completed out of 10 loss: 537607.225586 With Accuracy: 0.596623
Epoch 3 completed out of 10 loss: 428961.930908 With Accuracy: 0.618199
Epoch 4 completed out of 10 loss: 429376.342896 With Accuracy: 0.625704
Epoch 5 completed out of 10 loss: 162862.384064 With Accuracy: 0.582552
Epoch 6 completed out of 10 loss: 129290.723236 With Accuracy: 0.625704
Epoch 7 completed out of 10 loss: 150139.01918 With Accuracy: 0.58818
Epoch 8 completed out of 10 loss: 175318.36879 With Accuracy: 0.61257
Epoch 9 completed out of 10 loss: 48992.6575661 With Accuracy: 0.630394
Epoch 10 completed out of 10 loss: 13355.5717468 With Accuracy: 0.620075
Epoch 11 completed out of 10 loss: 11651.3329475 With Accuracy: 0.614447
Epoch 12 completed out of 10 loss: 8369.30546999 With Accuracy: 0.615385
Epoch 13 completed out of 10 loss: 9381.27617818 With Accuracy: 0.626642
Epoch 14 completed out of 10 loss: 9262.21038318 With Accuracy: 